# 房价预测


## NianGao

**2019.7**

**本kernel借鉴于以下三个写的很好的kernel**
- https://www.kaggle.com/pmarcelino/comprehensive-data-exploration-with-python#COMPREHENSIVE-DATA-EXPLORATION-WITH-PYTHON  
- https://www.kaggle.com/lavanyashukla01/how-i-made-top-0-3-on-a-kaggle-competition   
- https://www.kaggle.com/serigne/stacked-regressions-top-4-on-leaderboard    

# 前言
首先,在开始之前,我们要关注以下关键点:
- 这是一个回归任务还是一个分类任务?  回归任务
- 哪些特征是离散的,哪些特征是连续的?
- 房屋的售价分布情况如何?
- 是否存在缺失值?缺失值在训练集和测试集的分布是怎么样的?缺失值如何处理?
- 离散型变量如何处理?
- 使用交叉验证
- 使用模型融合
- 特征与特征之间的关系是怎么样的?特征与预测值之间的关系是怎么样的?
- 是否存在异常点,异常点如何处理?

# 一.数据预处理

## 1.  观察数据

首先,先简单观察数据
- 初步得出一些结论
- 粗略的了解属性的含义
- 观察样本数,特征数,空值等
 


首先我们导入必要的包并加载数据

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from pandas.core.dtypes.common import is_numeric_dtype
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import norm, skew

# 设置图片中文字符显示
plt.rcParams['font.family'] = ['sans-serif']
plt.rcParams['font.sans-serif'] = ['SimHei']

data_train = pd.read_csv('../input/train.csv')
data_test = pd.read_csv('../input/test.csv')

让我们简单的看一下数据

In [ ]:
data_train.head(5)

观察表格数据,我们可以发现
- 第一列是id,代表着数据索引
- 最后一列是SalePrice,是我们要预测的房价
- 其余大量的属性值的解释可以看这里 https://blog.csdn.net/Nyte2018/article/details/89977261
- 表格中存在着连续属性,离散属性和空值.

数据集全部特征如下:

In [ ]:
 data_train.columns.values

数据的shape如下:
- 注意: 因为有两列不是特征(id,SalePrice),实际特征数为81-2=79个

In [ ]:
data_train.shape

## 2. 分析"特征与特征之间的关系"

这里我们要知道,数据特征之间的一些基本知识
- 特征与特征之间的线性相关度尽可能小,因为如果两个特征线性相关,相当于是一个特征,即两个特征里有一个特征没有用出
- 特征与预测值的相关度尽可能大,因为相关度越高,代表它越是一个好特征

 ### 绘制散点图

让我们画出所有特征和SalePrice的散点图,看看是否能发现什么

In [ ]:
sns.set()

for i in range(9):
    sns.pairplot(data_train, x_vars=data_train.columns.values[10*i:10*(i+1)], y_vars="SalePrice")
plt.show()

这里可以发现很多有意思的规律,这里举几个例子

1. 可以观察到有些属性和预测值基本属于线性关系,这些属性是很好的属性:

 如:grlivarea,TotalBsmtSF等等
 
2. 当然有些离散型变量也有很强的规律性,这些也和预测值总体上呈线性关系

  如:OverallQual YearBuilt等等
  
 
3. 如Utilities属性,绝大部分的样本属性值都相同,说明该特征的用处不大

4. 如PoolQc属性,缺失值很多,图上只有几个点
 


如果知道每个特征的中文含义,从常识出发,我们也可以知道哪些特征对房价影响大.比如:

面积越大的房子越贵,新房子总体上要比旧房子贵,有车库的总体上比没车库的贵等等..

 ###  绘制特征矩阵

特征矩阵可以总体上看出特征之间的关系

In [ ]:
#correlation matrix
corrmat = data_train.corr()
corrmat.sort_values('SalePrice',ascending=False).index
f, ax = plt.subplots(figsize=(12, 12))
sns.heatmap(corrmat, vmax=.8, square=True);

特征太多了,看的眼花缭乱.
我们试着看看和预测值相关度最高的10个特征

In [ ]:
#saleprice correlation matrix
k = 10 #number of variables for heatmap
cols = corrmat.nlargest(k, 'SalePrice')['SalePrice'].index
# print(corrmat.sort_values('SalePrice',ascending=False).index)
cm = np.corrcoef(data_train[cols].values.T)
sns.set(font_scale=1.25)
hm = sns.heatmap(cm, cbar=True, annot=True, square=True, fmt='.2f', annot_kws={'size': 10}, yticklabels=cols.values, xticklabels=cols.values)
plt.show()

让我们观察图片

特征值与预测值之间的关系:
 - 图中数据值代表着与SalePrice的相关度,数值越大越好
 - 第一列数值由大到小下降,代表着其余9个特征和SalePrice的相关度也是由大到小下降的

特征值与特征值之间的关系
 - 可以发现 GarageCars和GarageArea的相关度很高,特征之间相关度很高不是一个好消息,我们可以删除掉多余特征,但也可以保留.
 - 与此相似,TotalBsmt和1stFlrSF相关度也很高

当然只观察10个特征还是太少了,我们把特征值按照相关度进行排序并打印出来看看

In [ ]:
print(corrmat.sort_values('SalePrice',ascending=False).index)

接下来我们要分析这其中最重要的一些特征

 ### 重要特征分析与异常值处理

看看GrLivArea与SalePrice的关系

In [ ]:
fig, ax = plt.subplots()
ax.scatter(x = data_train['GrLivArea'], y = data_train['SalePrice'])
plt.ylabel('SalePrice', fontsize=13)
plt.xlabel('GrLivArea', fontsize=13)
plt.show()

比较明显的异常点有:
- 右下角两个点,明显不符合规律,我们将其删除
- 右上角两个点,虽然不是很符合规律,但因为符合大体上的趋势,我们将其保留

注意: 删除掉极端的异常点和保留不是那么异常的点都是有好处的,可以增加程序的健壮性.

> 训练数据中可能还有其他异常值。但是，如果在测试数据中也存在异常值，那么删除所有异常值可能会严重影响我们的模型。
> 这就是为什么我们不将它们全部删除，而是设法使我们的一些模型在它们上面保持健壮。

In [ ]:
data_train = data_train.drop(data_train[(data_train['GrLivArea']>4000) & (data_train['SalePrice']<300000)].index) # 删除异常值


看看TotalBsmtSF与SalePrice的关系

In [ ]:
fig, ax = plt.subplots()
ax.scatter(x = data_train['TotalBsmtSF'], y = data_train['SalePrice'])
plt.ylabel('SalePrice', fontsize=13)
plt.xlabel('TotalBsmtSF', fontsize=13)
plt.show()

没有什么特别异常的点
- 有很多数值为0的点,但这些点很可能是缺失值,不属于异常点
- 我们可以尝试删除一些点（例如，totalbsmtsf>3000的点），但我认为这不值得,因为这些点也是符合整体趋势的

 看看OverallQual与SalePrice的关系

In [ ]:
 # 数字型离散变量
data = pd.concat([data_train['SalePrice'], data_train["OverallQual"]], axis=1)
f, ax = plt.subplots(figsize=(8, 6))
fig = sns.boxplot(x='OverallQual', y="SalePrice", data=data)
# fig.axis(ymin=0, ymax=800000);

对于离散型变量,我们使用箱形图来进行观察

> 箱形图: 最大值、最小值、中位数和两个四分位数；然后， 连接两个四分位数画出箱子；再将最大值和最小值与箱子相连接，中位数在箱子中间
> ![箱形图](https://gss0.bdstatic.com/94o3dSag_xI4khGkpoWK1HF6hhy/baike/c0%3Dbaike80%2C5%2C5%2C80%2C26/sign=4e5ee1bdacaf2eddc0fc41bbec796a8c/aa18972bd40735fade9ad1029e510fb30f240826.jpg)

可以看出,其上升趋势有些类似于指数形式

看看YearBuilt与SalePrice的关系

In [ ]:
 # 数字型离散变量
data = pd.concat([data_train['SalePrice'], data_train['YearBuilt']], axis=1)
f, ax = plt.subplots(figsize=(16, 8))
fig = sns.boxplot(x='YearBuilt', y="SalePrice", data=data)
fig.axis(ymin=0, ymax=800000);
plt.xticks(rotation=90);

上升趋势不是很明显,但总体是上升的

## 3.分析预测值"SalePrice"的分布规律

观察SalePrice的属性描述.



In [ ]:
 data_train['SalePrice'].describe()

从图表上很难看出些什么,让我们可视化以下数据
可视化观察SalePrice属性

In [ ]:
sns.distplot(data_train['SalePrice']);


可以发现数据的分布类似于正态分布,但该"正态分布"是一个偏斜的正态分布.
​
为了衡量数值的倾斜程度,这里有峰度和偏度两个概念
​
 > 峰度：峰度（Kurtosis）是描述某变量所有取值分布形态陡缓程度的统计量。
 > 正态分布的峰度为0
 > 若峰度>0，分布的峰态陡峭（高尖）；
 > 若峰度<0，分布的峰态平缓（矮胖）；
 
 >偏度：偏度（Skewness）是描述某变量取值分布对称性的统计量。
 >正态分布的偏度为0
 >偏度>0 长尾巴拖在右边。
 >偏度<0 长尾巴拖在左边。
 >同时偏度的绝对值越大，说明分布的偏移程度越严重。
 >
 >【注意】数据分布的左偏或右偏，指的是数值拖尾的方向，而不是峰的位置。
 
 让我们看一下它的偏度和峰度

In [ ]:
print("Skewness: %f" % data_train['SalePrice'].skew())
print("Kurtosis: %f" % data_train['SalePrice'].kurt())

为了使其更为正态分布,这里使用log转换来处理数据

重新观察数据

In [ ]:
data_train["SalePrice"] = np.log1p(data_train["SalePrice"])
sns.distplot(data_train['SalePrice'],fit=norm );

图中黑色的线是正态分布曲线,蓝色的线是拟合后的曲线

现在，歪斜得到了纠正，数据看起来更符合正态分布。

## 4. 缺失值分析与处理

### 缺失值分析

这里有个小**提示**:

在处理缺失值时,建议将所有数据全部进行分析,即将训练集和测试集的所有数据放在一起,组成全部的数据集,并分析数据的缺失情况

如果不这样做的话,就会出现有些属性在训练集没有缺失,但在测试集却出现了缺失,从而可能使程序出现异常.

接下来要做的是
- 合并训练集测试集数据.
- 删除'SalePrice',"Id"这两个非特征属性  

In [ ]:
ntrain = data_train.shape[0]
ntest = data_test.shape[0]
y_train = data_train.SalePrice.values
all_data = pd.concat((data_train, data_test)).reset_index(drop=True)
x_all = all_data.drop(['SalePrice',"Id"], axis=1, inplace=True) # 删除'SalePrice',"Id"属性  我
print("all_data size is : {}".format(all_data.shape))

计算缺失值的个数和比例

In [ ]:
all_data_na = (all_data.isnull().sum() / len(all_data)) * 100
all_data_na = all_data_na.drop(all_data_na[all_data_na == 0].index).sort_values(ascending=False)[:30]
missing_data = pd.DataFrame({'Missing Ratio' :all_data_na})
missing_data.head()  # 这里只显示了5条

可以看出,有些值缺失的特别多,但表格还不够直观,接下来我们使用进行可视化分析

可视化分析缺失值

In [ ]:
f, ax = plt.subplots(figsize=(15, 12))
plt.xticks(rotation='90')
sns.barplot(x=all_data_na.index, y=all_data_na)
plt.xlabel('Features', fontsize=15)
plt.ylabel('Percent of missing values', fontsize=15)
plt.title('Percent missing data by feature', fontsize=15)

### 缺失值处理

对于缺失值的处理:
- 字符类型的特征填补None,数字类型的特征填补0,代表一种新的分类,即表示没有,如PoolQC填补None代表没有泳池.
- 如果某个特征缺失值特别少,可以删除含有缺失值的行,即删除掉该特征缺失的样本
- 根据现实语义填补缺失值,比如姓名中为Mrs的在性别中填补女
- 使用平均数进行填补,不过这么做不是很好的选择
- 对于某个特征,如果某个类别特别多,可以使用众数填补,比如:性别列80%为男,20%为女,缺失值可以填补为男
- 使用随机森林模型填补,寒小阳的泰坦尼克的博客就这么做过

进行缺失值填补:

In [ ]:
for i in ["PoolQC", "MiscFeature","Alley","Fence","FireplaceQu"]:
    all_data[i] = all_data[i].fillna("None")
print(all_data.shape)

 LotFrontage,其含义为街道的面积,
 
 根据现实含义,在Neighborhood相同的情况下,LotFrontage很有可能也相同,
 
 即邻居相同的人街道也可能相同,所以这里对邻居分组,用每组内的LotFrontage中位数填补缺失值

In [ ]:
all_data["LotFrontage"] = all_data.groupby("Neighborhood")["LotFrontage"].transform(lambda x: x.fillna(x.median()))

对于接下来5个GarageX数据,填充None值

In [ ]:
for col in ('GarageType', 'GarageFinish', 'GarageQual', 'GarageCond','GarageYrBlt'):
    all_data[col] = all_data[col].fillna('None')

对于接下来5个Bsmt数据,填充None值

In [ ]:
for col in ('BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2'):
    all_data[col] = all_data[col].fillna('None')

对于接下来两个MasVn,仍然填充none

In [ ]:
all_data["MasVnrType"] = all_data["MasVnrType"].fillna("None")
all_data["MasVnrArea"] = all_data["MasVnrArea"].fillna(0)

对于损失极少的特征,可以使用众数填补

In [ ]:
for col in ('MSZoning', 'Functional', 'Electrical', 'GarageCond','Exterior1st','Exterior2nd','SaleType',"KitchenQual" ):
    all_data[col] = all_data[col].fillna(all_data[col].mode()[0])
    print(all_data[col].mode()[0])
print(all_data.shape)

仍然使用0填补

In [ ]:
for col in ('BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF','TotalBsmtSF', 'BsmtFullBath', 'BsmtHalfBath'):
    all_data[col] = all_data[col].fillna(0)

对于GarageArea和GarageCars使用0填补,代表没有Garage

In [ ]:
for col in ( 'GarageArea', 'GarageCars'):
    all_data[col] = all_data[col].fillna(0)
print(all_data.shape)

删除Utilities列,因为他的值几乎一样,值几乎一样的列存在的意义不大

In [ ]:
all_data = all_data.drop(['Utilities'], axis=1)
print(all_data.shape)

检查是否有缺失值

In [ ]:
#Check remaining missing values if any 
all_data_na = (all_data.isnull().sum() / len(all_data)) * 100
all_data_na = all_data_na.drop(all_data_na[all_data_na == 0].index).sort_values(ascending=False)
missing_data = pd.DataFrame({'Missing Ratio' :all_data_na})
missing_data.head()

### 5. 特征处理

这里我们要可以尝试如下工作
- 是否可以添加新的特征?
- 特征的倾斜程度如何?怎样解决?
- 对离散型数据进行编码,如hot编码

将数值类型的离散型变量转化为字符串类型,在这之后可以对这些特征进行hot编码

具体的编码方式可以百度搜索

这里举个例子
> hot编码: 对性别编码  编码后:  男 0 1  女 1 0
> 另一种编码: 男 0 女 1 

另一种编码比起hot编码,会对男女两个特征值有大小之分,而hot编码没有.但hot编码会产生大量的特征.

In [ ]:

all_data['MSSubClass'] = all_data['MSSubClass'].apply(str)


#Changing OverallCond into a categorical variable
all_data['OverallCond'] = all_data['OverallCond'].astype(str)


#Year and month sold are transformed into categorical features.
all_data['YrSold'] = all_data['YrSold'].astype(str)
all_data['MoSold'] = all_data['MoSold'].astype(str)

 ### 添加特征

添加一个特征
 - 由于与面积相关的特征对于确定房价非常重要，我们又增加了一个特征，即地下室总面积、每栋房子的一层和二层面积


In [ ]:
all_data['TotalSF'] = all_data['TotalBsmtSF'] + all_data['1stFlrSF'] + all_data['2ndFlrSF']

 ### 解决连续型分布的倾斜特征

这里的特征处理方式与之前处理SalePrice类似,不过我们这里不使用log变换,而是使用Box Cox变换

获得特征的偏度

In [ ]:
numeric_feats = all_data.dtypes[ all_data.dtypes != "object"].index

# Check the skew of all numerical features
skewed_feats = all_data[numeric_feats].apply(lambda x: skew(x.dropna())).sort_values(ascending=False)
print("\nSkew in numerical features: \n")
skewness = pd.DataFrame({'Skew' :skewed_feats})
skewness.head(10)

我们可以认为当skewness>0.75时的特征就需要转换

使用Box Cox 转换skewness>0.75的特征

In [ ]:
from scipy.special import boxcox1p
from scipy.stats import boxcox_normmax

skewness = skewness[abs(skewness) > 0.75]
print("There are {} skewed numerical features to Box Cox transform".format(skewness.shape[0]))



for i in skewness.index:
    all_data[i] = boxcox1p(all_data[i], boxcox_normmax(all_data[i] + 1))
print(all_data.shape)

### 对离散型变量进行hot编码

In [ ]:
all_data = pd.get_dummies(all_data).reset_index(drop=True)
print(all_data.shape)

 ### 重新创建训练和测试集

In [ ]:
X_train = all_data.iloc[:ntrain]
X_test = all_data.iloc[ntrain :]
print(X_train.shape,X_test.shape,y_train.shape)

# 二.交叉验证,模型建立与模型融合

导入所需包

In [ ]:
from sklearn.linear_model import ElasticNet, Lasso,  BayesianRidge, LassoLarsIC
from sklearn.ensemble import RandomForestRegressor,  GradientBoostingRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.metrics import mean_squared_error
import xgboost as xgb
import lightgbm as lgb

## 1. 交叉验证

 ### 定义交叉验证策略

 我们使用sklearn的cross-val-score函数。但是，这个函数没有shuffle属性，我们添加一行代码，以便在交叉验证之前对数据集进行shuffle操作。
 
 即: 数据在交叉验证之前必须要打散,即随机组合这些数据,默认的cross-val-score不具备打散功能.
 
 这里使用12折交叉验证

In [ ]:
n_folds = 12
kf = KFold(n_splits=12, random_state=42, shuffle=True)
def rmsle(y, y_pred):
    return np.sqrt(mean_squared_error(y, y_pred))


def cv_rmse(model, X=X_train):
    rmse = np.sqrt(-cross_val_score(model, X_train.values, y_train,
                                    scoring="neg_mean_squared_error", cv=kf))
    return (rmse)


## 2.建模

- 这里的模型我也不知道为什么选择了这些

- 包括模型的参数我也不清楚是如何选择的

- xgboost,lightgbm,StackingCVRegressor这三个我也没有搞清楚原理,这里先暂且略过,只知道StackingCVRegressor是一种叠加模型.


注: 

对于模型RidgeCV,SVR,ElasticNet,Lasso要进行数据预处理,

类似于数据归一化,这里使用RobustScaler,RobustScaler可以使模型具有更强的健壮性

In [ ]:
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from sklearn.linear_model import Ridge, RidgeCV
from sklearn.svm import SVR
from mlxtend.regressor import StackingCVRegressor
lightgbm = LGBMRegressor(objective='regression', 
                       num_leaves=6,
                       learning_rate=0.01, 
                       n_estimators=7000,
                       max_bin=200, 
                       bagging_fraction=0.8,
                       bagging_freq=4, 
                       bagging_seed=8,
                       feature_fraction=0.2,
                       feature_fraction_seed=8,
                       min_sum_hessian_in_leaf = 11,
                       verbose=-1,
                       random_state=42)

# XGBoost Regressor
xgboost = XGBRegressor(learning_rate=0.01,
                       n_estimators=6000,
                       max_depth=4,
                       min_child_weight=0,
                       gamma=0.6,
                       subsample=0.7,
                       colsample_bytree=0.7,
                       objective='reg:squarederror',
                       nthread=-1,
                       scale_pos_weight=1,
                       seed=27,
                       reg_alpha=0.00006,
                       random_state=42)

# Ridge Regressor
ridge_alphas = [1e-15, 1e-10, 1e-8, 9e-4, 7e-4, 5e-4, 3e-4, 1e-4, 1e-3, 5e-2, 1e-2, 0.1, 0.3, 1, 3, 5, 10, 15, 18, 20, 30, 50, 75, 100]
ridge = make_pipeline(RobustScaler(), RidgeCV(alphas=ridge_alphas, cv=kf))

# Support Vector Regressor
svr = make_pipeline(RobustScaler(), SVR(C= 20, epsilon= 0.008, gamma=0.0003))

ENet = make_pipeline(RobustScaler(), ElasticNet(alpha=0.0005, l1_ratio=.9, random_state=3))

lasso = make_pipeline(RobustScaler(), Lasso(alpha =0.0005, random_state=1))

stack_gen = StackingCVRegressor(regressors=(xgboost, lightgbm, svr, ridge, ENet, lasso),
                                meta_regressor=xgboost,
                                use_features_in_secondary=True)

进行交叉验证,并打分

这里我将有些模型的交叉验证的代码注释掉,因为运行起来好慢.



In [ ]:
scores = {}
# # 慢
# score = cv_rmse(lightgbm)
# print("lightgbm: {:.4f} ({:.4f})".format(score.mean(), score.std()))
# scores['lgb'] = (score.mean(), score.std())

In [ ]:
# score = cv_rmse(xgboost)
# # 很慢
# print("xgboost: {:.4f} ({:.4f})".format(score.mean(), score.std()))
# scores['xgb'] = (score.mean(), score.std())

In [ ]:
score = cv_rmse(svr)
print("SVR: {:.4f} ({:.4f})".format(score.mean(), score.std()))
scores['svr'] = (score.mean(), score.std())

In [ ]:
# # 比较慢, 要进行网格搜索,
# score = cv_rmse(ridge)
# print("ridge: {:.4f} ({:.4f})".format(score.mean(), score.std()))
# scores['ridge'] = (score.mean(), score.std())

In [ ]:
score = cv_rmse(ENet)
print("ENet: {:.4f} ({:.4f})".format(score.mean(), score.std()))
scores['ENet'] = (score.mean(), score.std())

In [ ]:
score = cv_rmse(lasso)
print("lasso: {:.4f} ({:.4f})".format(score.mean(), score.std()))
scores['lasso'] = (score.mean(), score.std())

## 3. 预测

In [ ]:
# 超级慢
print('stack_gen')
stack_gen_model = stack_gen.fit(np.array(X_train), np.array(y_train))

In [ ]:
# 慢
print('lightgbm')
lgb_model_full_data = lightgbm.fit(X_train, y_train)

In [ ]:
# 慢
print('xgboost')
xgb_model_full_data = xgboost.fit(X_train, y_train)

In [ ]:
print('Svr')
svr_model_full_data = svr.fit(X_train, y_train)

In [ ]:
print('Ridge')
ridge_model_full_data = ridge.fit(X_train, y_train)

In [ ]:
print('ENet')
rf_model_full_data = ENet.fit(X_train, y_train)

In [ ]:
print('lasso')
gbr_model_full_data = lasso.fit(X_train, y_train)

对最终预测的模型得分进行一个加权

这里的权重我也不知道怎么计算出来的

In [ ]:
def blended_predictions(X):
    return ((0.1 * ridge_model_full_data.predict(X)) + \
            (0.2 * svr_model_full_data.predict(X)) + \
            (0.1 * gbr_model_full_data.predict(X)) + \
            (0.1 * xgb_model_full_data.predict(X)) + \
            (0.1 * lgb_model_full_data.predict(X)) + \
            (0.05 * rf_model_full_data.predict(X)) + \
            (0.35 * stack_gen_model.predict(np.array(X))))

最终模型得分

In [ ]:
blended_score = rmsle(y_train, blended_predictions(X_train))
scores['blended'] = (blended_score, 0)
print('RMSLE score on train data:')
print(blended_score)

## 4.输出结果

In [ ]:
submission = pd.read_csv("../input/sample_submission.csv")
submission.shape
submission.head()

In [ ]:
submission.iloc[:,1] = np.floor(np.expm1(blended_predictions(X_test)))
submission.head()

In [ ]:
submission.to_csv("submission_regression.csv", index=False)